<DIV ALIGN=CENTER>

# Introduction to Social Media: Email
## Professor Robert J. Brunner
  
</DIV>  
-----
-----


## Introduction

In this IPython Notebook, we explore 

Read and process email messages.

Discuss how to send/recieve email?

Spam/Ham classification? Enron/SPam Assasin?

Clinton emails?

-----


-----

## Student Activity

In the preceding cells, we used . Now that you
have run the Notebook, go back and make the following changes to see how
the results change.

1. Change the ?
2. Change the ?
3. Try making a ?

Finally, try applying a (discuss in class forums?

-----

-----

### Email Text Parsing

Python provides built-in support for [processing email messages][pem],
which are an often overlooked source of information in data science
projects. The library is part of the core Python distribution, and
includes support for parsing email messages, as well as sending and
receiving emails. For our purpose, we simply need to read in text and
create an email `message`, which provides access to the basic email
contents. The `message` instance provides access to the email header
information as well as any payload data. 

Normally the payload is the email message, but with multipart messages,
like HTML email messages, an email can have multiple payloads. In the
next several code cells, we create an email `message` by reading an
email from a file (the email should look familiar). We subsequently
explore the Python email message interface to extract email headers and
the message payload, before grabbing the HTML message for later parsing.

-----

[pem]: https://docs.python.org/3/library/email.html

In [1]:
import email as em
from email import policy

with open("data/ham/prg.eml") as fin:
    msg = em.message_from_file(fin, policy=policy.default)

import pprint
pp = pprint.PrettyPrinter(indent=2, depth=2, width=80, compact=True)

pp.pprint(msg.keys())

[ 'Received', 'Received', 'Received', 'Received', 'Received', 'Date', 'To',
  'From', 'Subject', 'Message-ID', 'X-Mailer', 'List-Id', 'List-Help',
  'X-Course-Id', 'X-Course-Name', 'Precedence', 'X-Auto-Response-Suppress',
  'Auto-Submitted', 'Content-Type', 'X-Spam-Reason', 'Return-Path',
  'X-MS-Exchange-Organization-AuthSource', 'X-MS-Exchange-Organization-AuthAs',
  'X-MS-Exchange-Organization-Antispam-Report',
  'X-MS-Exchange-Organization-SCL',
  'X-MS-Exchange-Organization-AVStamp-Mailbox', 'MIME-Version']


In [2]:
print('To:', msg['to'])
print('From:', msg['from'])
print('Subject:', msg['subject'])

To: Robert Brunner <xxxyyy@illinois.edu>
From: Robert Brunner <xxxyyy@illinois.edu>
Subject: INFO 490 RB2 SP16: Peer Review Grading


In [3]:
print(msg.as_string()[2340:2693])

-------------------------------------------------------------
Peer Review Grading
by Robert Brunner - Monday, 29 February 2016, 10:22 AM
---------------------------------------------------------------------
I have received several emails or private Moodle messages regarding peer
grading. The basic context seems to be concern about losing a few points



In [4]:
pp.pprint([att for att in dir(msg) if '__' not in att])

[ '_add_multipart', '_body_types', '_charset', '_default_type', '_find_body',
  '_get_params_preserve', '_headers', '_make_multipart', '_payload',
  '_unixfrom', 'add_alternative', 'add_attachment', 'add_header', 'add_related',
  'as_bytes', 'as_string', 'attach', 'clear', 'clear_content', 'defects',
  'del_param', 'epilogue', 'get', 'get_all', 'get_body', 'get_boundary',
  'get_charset', 'get_charsets', 'get_content', 'get_content_charset',
  'get_content_disposition', 'get_content_maintype', 'get_content_subtype',
  'get_content_type', 'get_default_type', 'get_filename', 'get_param',
  'get_params', 'get_payload', 'get_unixfrom', 'is_attachment', 'is_multipart',
  'items', 'iter_attachments', 'iter_parts', 'keys', 'make_alternative',
  'make_mixed', 'make_related', 'policy', 'preamble', 'raw_items',
  'replace_header', 'set_boundary', 'set_charset', 'set_content',
  'set_default_type', 'set_param', 'set_payload', 'set_raw', 'set_type',
  'set_unixfrom', 'values', 'walk']


In [5]:
# Display part of the message

if msg.is_multipart() == True:
    data = msg.get_payload(0)
    html=msg.get_payload(1)

    print("Text Data:\n---------\n", data.get_content()[:941])

Text Data:
---------
 
INFO 490 RB2 SP16 -> Forums -> Announcements -> Peer Review Grading
https://learn.illinois.edu/mod/forum/discuss.php?d=938611
---------------------------------------------------------------------
Peer Review Grading
by Robert Brunner - Monday, 29 February 2016, 10:22 AM
---------------------------------------------------------------------
I have received several emails or private Moodle messages regarding peer
grading. The basic context seems to be concern about losing a few points
from one reviewer. 

To be clear, we do not intervene in Peer Assessment unless there has ben an
egregious violation. An example of this would be someone giving a zero and
saying 'The code does not run' when it clearly does run. Or 'The code is
empty' when the assignment was completed. Losing a few points occasionally
will not lower your grade. And other than write the best code you possibly
can and document it thoroughly, we do not have any advice!


In [6]:
# Grab content, as decoded text
from IPython.display import HTML
HTML(html.get_content())

,"Peer Review Gradingby Robert Brunner - Monday, 29 February 2016, 10:22 AM"
,"I have received several emails or private Moodle messages regarding peer grading. The basic context seems to be concern about losing a few points from one reviewer. To be clear, we do not intervene in Peer Assessment unless there has ben an egregious violation. An example of this would be someone giving a zero and saying 'The code does not run' when it clearly does run. Or 'The code is empty' when the assignment was completed. Losing a few points occasionally will not lower your grade. And other than write the best code you possibly can and document it thoroughly, we do not have any advice! The point of peer review is two-fold: 1) You get a chance to see how your peers completed an assignment. In some case, you might learn how to improve your own code, learn new coding tricks, or see ways that your coding style could improve. 2) You have your code reviewed by your peers and obtain comments from them that (hopefully) will be constructive and help you improve your work. In some cases, a review might grade you lower than you like, or in seeming contrast to others. Learning to deal with situations like this is part of the course. You will (probably) experience many of these situations later in your career, either via an interview (which is a form of peer assessment), in the workplace with pair coding or code reviews, or just via general interactions at conferences or public meetings. And if you ever release code online, expect comments and criticisms! Finally, please remember that this is a large online course, which Edward and I are building as the course is being offered. Please refrain from frivolous questions or peer assessment regrade requests; otherwise, we will be forced to invoke point reductions: https://github.com/UI-DataScience/info490-sp16/blob/master/orientation/syllabus.md#point-reductions and also the special point reduction under the Peer Review section: https://github.com/UI-DataScience/info490-sp16/blob/master/orientation/syllabus.md#peer-review Having said this, please understand we encourage valid questions; we do not want to stifle the learning process. Robert ReplySee this post in context"


In [7]:
# Get raw payload, undecoded
print(html.get_payload()[:115])

print(20*'-')
print('Payload parameters: \n', html.get_params())

<html><head>
<meta http-equiv=3D"Content-Type" content=3D"text/html; charset=3Dutf-8"></=
head>
<body id=3D"email">
--------------------
Payload parameters: 
 [('text/html', ''), ('charset', 'UTF-8')]


-----

### Spam Classification

-----

In [8]:
# https://spamassassin.apache.org/publiccorpus/readme.html
# Grabbed spam_2 and easyham_2

import os

mypath = '/home/data_scientist/rppdm/data/email/'

ham = []
spam = []

for root, dirs, files in os.walk(os.path.join(mypath, 'ham')):
    for file in files:
        with open(os.path.join(root, file), encoding='ISO-8859-1') as fin:
            msg = em.message_from_file(fin, policy=policy.default)
            for part in msg.walk():
                if part.get_content_type() == 'text/plain':
                    data = part.get_payload(None, decode=True)

            ham.append(data.decode(encoding='ISO-8859-1'))


for root, dirs, files in os.walk(os.path.join(mypath, 'spam')):
    for file in files:
        with open(os.path.join(root, file), encoding='ISO-8859-1') as fin:
            msg = em.message_from_file(fin, policy=policy.default)
            for part in msg.walk():
                if part.get_content_type() == 'text/plain':
                    data = part.get_payload(None, decode=True)#.get_content()

            spam.append(data.decode(encoding='ISO-8859-1'))

print('{0} ham messages.'.format(len(ham)))
print('{0} spam messages.'.format(len(spam)))

1401 ham messages.
1397 spam messages.


In [9]:
import numpy as np

pos_emails = np.array(ham)
neg_emails = np.array(spam) 

pos_labels = np.ones(pos_emails.shape[0])
neg_labels = np.zeros(neg_emails.shape[0])

print('{0} Good Emails'.format(pos_emails.shape[0]))
print('{0} Bad Emails'.format(neg_emails.shape[0]))

1401 Good Emails
1397 Bad Emails


In [10]:
split_value = 200

x_train = np.concatenate((pos_emails[:split_value], 
                          neg_emails[:split_value]), axis = 0)

x_test = np.concatenate((pos_emails[split_value:],
                         neg_emails[split_value:]), axis = 0)

y_train = np.concatenate((pos_labels[:split_value], 
                          neg_labels[:split_value]), axis = 0)

y_test = np.concatenate((pos_labels[split_value:],
                         neg_labels[split_value:]), axis = 0)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics

tools = [('cv', CountVectorizer()), ('nb', MultinomialNB())]
pclf = Pipeline(tools)


# Lowercase and restrict ourselves to about half the available features
pclf.set_params(cv__stop_words = 'english', \
                cv__ngram_range=(1,2), \
                cv__lowercase=True)

pclf.fit(x_train, y_train)
y_pred = pclf.predict(x_test)
print(metrics.classification_report(y_test, y_pred, target_names = ['Ham', 'Spam']))

             precision    recall  f1-score   support

        Ham       0.83      0.92      0.88      1197
       Spam       0.91      0.82      0.86      1201

avg / total       0.87      0.87      0.87      2398



-----

### Classify our email

-----

In [12]:
mypath = 'data'

print('Ham File Processing:')
print(20*'-')
for root, dirs, files in os.walk(os.path.join(mypath, 'ham')):
    for file in files:
        with open(os.path.join(root, file), encoding='ISO-8859-1') as fin:
            msg = em.message_from_file(fin, policy=policy.default)
            for part in msg.walk():
                if part.get_content_type() == 'text/plain':
                    data = part.get_payload(None, decode=True)
                lst = []
                lst.append(data.decode(encoding='ISO-8859-1'))
                value = pclf.predict(np.array(lst))
                if value[0] == 0:
                    my_pred = 'spam'
                else:
                    my_pred = 'ham'

            print('{0} classified as {1}'.format(file, my_pred))

print(20*'-')          
print('Spam File Processing:')
print(20*'-')
for root, dirs, files in os.walk(os.path.join(mypath, 'spam')):
    for file in files:
        with open(os.path.join(root, file), encoding='ISO-8859-1') as fin:
            msg = em.message_from_file(fin, policy=policy.default)
            for part in msg.walk():
                if part.get_content_type() == 'text/plain':
                    data = part.get_payload(None, decode=True)
                lst = []                    
                lst.append(data.decode(encoding='ISO-8859-1'))
                value = pclf.predict(np.array(lst))
                if value[0] == 0:
                    my_pred = 'spam'
                else:
                    my_pred = 'ham'

            print('{0} classified as {1}'.format(file, my_pred))

Ham File Processing:
--------------------
kmdb.eml classified as ham
prg.eml classified as ham
pvc.eml classified as ham
rw6.eml classified as ham
rw7.eml classified as ham
w7.eml classified as ham
--------------------
Spam File Processing:
--------------------
add.eml classified as spam
cnn.eml classified as spam
lcm.eml classified as spam
mws.eml classified as spam
prs.eml classified as spam
vsfih.eml classified as spam


-----

## Student Activity

In the preceding cells, we used developed a simple Naive Bayes spam classifier. Now that you have run the Notebook, go back and make
the following changes to see how the results change.

1. Change split_value, try lower and higher values. How does the validation sample perform? How does our blind data perform?

Try other techniques.

Why does it perform the way it does on our Blind data?


-----